In [1]:
%matplotlib inline

In [2]:
import torch
import matplotlib.pyplot as plt
import numpy as np

In [3]:
# get truth and iamges
ds = torch.load("../models/simulated_blended_galaxies.pt")

images = ds.pop("images")
background = ds.pop("background")
images.shape

from bliss.catalog import TileCatalog 
tile_catalog = TileCatalog(4, {**ds})
truth = tile_catalog.to_full_params()

In [8]:
# get predictions 
from case_studies.galsim_galaxies.get_figures import _load_models
from hydra import compose, initialize
with initialize(config_path="../config", version_base=None):
    cfg = compose("config", overrides={})

_, slen, _ = background.shape
bkg = background.expand(images.shape[0], 1, slen, slen)
device = torch.device("cuda:0")
encoder, decoder = _load_models(cfg, device)
tile_est = encoder.variational_mode(images, bkg)
tile_est = tile_est.cpu()
est = tile_est.to_full_params()


Global seed set to 42
Encoding ptiles: 1024it [00:51, 19.87it/s]


In [4]:
ordn = np.argsort(-truth.n_sources)

In [84]:
# idx = 133, 
fig, ax = plt.subplots(1, 1, figsize=(10, 10))

# 22, 33, 106
idx1 = 32
trim = 20
idx = ordn[idx1]
im = images[idx, 0, trim:-trim, trim:-trim].log().numpy()

# # truth
plocs1 = truth.plocs[idx]
plocs2 = est.plocs[idx]
for ii in range(truth.n_sources[idx].item()):
    ax.scatter(plocs1[ii, 1] + 24 - trim  - 0.5, plocs1[ii, 0] + 24 - trim - 0.5, c="r", s=100, marker='x')
# for ii in range(est.n_sources[idx].item()):
#     ax.scatter(plocs2[ii, 1] + 24 - 0.5, plocs2[ii, 0] + 24 - 0.5, c="b", s=50, marker='+')

ax.imshow(im, cmap="gray")
print(truth.n_sources[idx].item())

fig.savefig("example-blend.pdf", bbox_inches="tight")
plt.close()

8
